In [82]:
import re
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from glob import glob

In [5]:
import pickle
import os

In [6]:
def get_var_name(var_value, namespace):
    """
    Given a variable value and namespace (e.g. globals() or locals()),
    returns the name of the variable as a string.
    """
    for name in namespace:
        if namespace[name] is var_value:
            return name
    return None

def remove_field(element):
    element.send_keys(Keys.CONTROL + 'A')
    element.send_keys(Keys.DELETE)

In [7]:
path = 'driver/chromedriver.exe'
driver = webdriver.Chrome(path)

In [8]:
driver.maximize_window() 
driver.minimize_window() 
driver.maximize_window() 
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)

In [9]:
driver.implicitly_wait(5)
driver.get('https://www.jobstreet.co.id/')

Search Bar

In [213]:
'''next'''
job_name = 'relation'

job_title = '-'.join(job_name.split())
driver.get(f'https://www.jobstreet.co.id/id/job-search/{job_title}-jobs-in-indonesia/')
driver.implicitly_wait(20)

jobs = []
job_num = 1
job_idx = 0
total_data = driver.find_element_by_css_selector('[data-automation="searchResultBar"]').text.split()[-2]
print('Total', total_data, 'data.')

Total 594 data.


In [214]:
print('No.'.center(5), '[Scraped, Page, Total]'.center(25), 'Job Title'.center(60), 'Info')

while True:
    job_cards = driver.find_elements_by_css_selector('[data-search-sol-meta]')

    try:
        curr_page = int(driver.find_element_by_id('pagination').get_attribute('value'))
        driver.implicitly_wait(2)
    except:
        curr_page = 1

    for job_card in job_cards:
        job = {}

        # scroll
        driver.execute_script('arguments[0].scrollIntoView({block: "end"});', job_card)

        try:
            title = job_card.text.split('\n')[0]
            title_class = driver.find_element_by_xpath(f"//span[contains(text(), '{title}')]").get_attribute('class').split()

            if len(title_class) > 1:
                job_idx += 1
                # print('❌❌❌ Continued!')
                continue
        except:
            pass

        try:
            job_card.find_element_by_css_selector('[rel]').click()
            driver.implicitly_wait(10)
        except:
            job_card.find_element_by_css_selector('[rel]').click()
            driver.implicitly_wait(10)
        

        '''scraping'''
        # job header: list of header information
        try:
            job_header = driver.find_element_by_css_selector('[data-automation="jobDetailsHeader"]').text.split('\n')[:3]
            driver.implicitly_wait(5)

            job['Title'], job['Company'], job['Area'] = job_header
        except:
            print('❌ JobTitle raise exception!')
            job['Title'], job['Company'], job['Area'] = None, None, None

        # job detail object
        try:
            job_detail = driver.find_element_by_css_selector('[data-automation="splitModeJobDetailsScrollWrapper"]')
            driver.implicitly_wait(5)

            try:
                # job description: list of description
                job_description = driver.find_element_by_css_selector('[data-automation="jobDescription"]').text.split('\n')
                driver.implicitly_wait(5)

                job['Description'] = job_description
            except:
                print('❌ JobDescription raise exception!')
                job['Description'] = None

            try:
                # additional information: list of information
                job_additional_information = job_detail.find_element_by_xpath('div/div[2]/div/div[1]/div/div[2]').text.split('\n')[1:][1::2]
                driver.implicitly_wait(5)

                job_additional_information = [i for i in job_additional_information if i.lower() != 'tidak terspesifikasi']

                job['Information'] = job_additional_information
            except:
                print('❌ JobInformation raise exception!')
                job['Information'] = None
        except:
            print('❌ JobDescription and JobInformation raise exception!')
            job['Description'], job['Information'] = None, None

        jobs.append(job)

        # log
        print(f"{job_idx}.".ljust(5), f"[{job_num}, {curr_page}, {total_data}]".center(25), f"{job['Title']}".center(60), 'scraped successfully ... 😊👍')

        job_num += 1
        job_idx += 1
    
    try:
        # scroll to "lanjut"
        driver.execute_script('arguments[0].scrollIntoView({block: "center"});', driver.find_element_by_xpath("//span[contains(text(), 'Lanjut')]"))
    except:
        print('> raise break when scroll <')
        break

    # go to the next page
    driver.find_element_by_xpath("//span[contains(text(), 'Lanjut')]").click()
    driver.implicitly_wait(10)

    next_page = int(driver.find_element_by_id('pagination').get_attribute('value'))

    if curr_page == next_page:
        print('> raise break <')
        break


 No.    [Scraped, Page, Total]                           Job Title                           Info
0.           [1, 1, 594]                          Guest Relations Officer                    scraped successfully ... 😊👍
1.           [2, 1, 594]                         CUSTOMER RELATION OFFICER                   scraped successfully ... 😊👍
9.           [3, 1, 594]                         Customer Relations Manager                  scraped successfully ... 😊👍
12.          [4, 1, 594]                Marketing Officer (Patient Relation Officer)         scraped successfully ... 😊👍
14.          [5, 1, 594]         Customer Relation Officer (Primme Health & Beauty Clinic)   scraped successfully ... 😊👍
21.          [6, 1, 594]                         Tenant Relation Supervisor                  scraped successfully ... 😊👍
22.          [7, 1, 594]                  Investor Relations and Corporate Finance           scraped successfully ... 😊👍
24.          [8, 1, 594]                       Legal an

In [215]:
len(jobs)

107

In [216]:
filename = 'jobstreet_' + ''.join(job_name.title().split())

with open(f'data/scraped/{filename}.pickle', 'wb') as file:
    pickle.dump(jobs, file)

In [217]:
filepaths = glob('data/scraped/jobstreet_*.pickle')

filenames = [filepath.split('.')[0].split('_')[-1].lower() for filepath in filepaths]

if ''.join(job_name.lower().split()) in filenames:
    print(job_name.upper(), 'saved!')

RELATION saved!


In [474]:
# with open("data/scraped/jobstreet_Akuntansi.pickle", "rb") as f:
#     my_list = pickle.load(f)

# print(len(my_list))

3198


In [232]:
# body = driver.find_element_by_tag_name("body")
# body.send_keys(Keys.CONTROL + '-')

In [233]:
# job_search = driver.find_element_by_css_selector('#global-nav-typeahead > input')

# job_search.click()
# job_search.send_keys(Keys.CONTROL + 'a')
# job_search.send_keys(Keys.DELETE)
# job_search.send_keys('ios developer')

# location_search = driver.find_element_by_css_selector('#jobs-search-box-location-id-ember1360')

# location_search.click()
# location_search.send_keys(Keys.CONTROL + 'a')
# location_search.send_keys(Keys.DELETE)
# location_search.send_keys('indonesia')

# search_button = driver.find_element_by_xpath('//*[@id="global-nav-search"]/div/div[2]/button[1]')
# search_button.click()

# actions = ActionChains(driver)
# actions.send_keys(Keys.ENTER)
# actions.perform()